# GreenNode Rerank

>[GreenNode](https://greennode.ai/) is a global AI solutions provider and a **NVIDIA Preferred Partner**, delivering full-stack AI capabilities—from infrastructure to application—for enterprises across the US, MENA, and APAC regions. Operating on **world-class infrastructure** (LEED Gold, TIA‑942, Uptime Tier III), GreenNode empowers enterprises, startups, and researchers with a comprehensive suite of AI services

This notebook provides a walkthrough on getting started with the `GreenNodeRerank` retriever. It enables you to perform document search using built-in connectors or by integrating your own data sources, leveraging GreenNode's reranking capabilities for improved relevance.

## Overview

The `GreenNodeRerank` class enables integration with GreenNode Serverless AI’s reranking models via [LangChain](https://www.langchain.com/). These models are designed to rescore and reorder search results based on semantic relevance, making them ideal for enhancing the accuracy of information retrieval systems.

### Integration details

- **Provider**: [GreenNode Serverless AI](https://aiplatform.console.greennode.ai/playground)
- **Model Types**: Reranking models
- **Primary Use Case**: Reranking search results based on semantic relevance
- **Available Models**: Includes [BAAI/bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3) and other high-performance reranking models
- **Scoring**: Returns relevance scores used to reorder document candidates based on query alignment

## Setup

### Installation

The GreenNode integration can be installed via pip:

In [ ]:
%pip install --upgrade langchain-greennode

### Credentials

GreenNode requires an API key for authentication, which can be provided either as the `api_key` parameter during initialization or set as the environment variable `GREENNODE_API_KEY`. You can obtain an API key by registering for an account on [GreenNode Serverless AI](https://aiplatform.console.greennode.ai/playground).

In [ ]:
import getpass
import os

# Make sure you've set your API key as an environment variable
if "GREENNODE_API_KEY" not in os.environ:
    os.environ["GREENNODE_API_KEY"] = getpass.getpass("Enter your GreenNode API key: ")

## Instantiation

The `GreenNodeRerank` class can be instantiated with optional parameters for the API key and model name:

In [ ]:
from langchain_greennode import GreenNodeRerank

# Initialize the embeddings model
reranker = GreenNodeRerank(
    # api_key="YOUR_API_KEY",  # You can pass the API key directly
    model="BAAI/bge-reranker-v2-m3",  # The default embedding model
    top_n=3
)

### Available Models

The list of supported models is available at https://aiplatform.console.greennode.ai/models

## Reranking Search Results

Reranking models enhance retrieval-augmented generation (RAG) workflows by refining and reordering initial search results based on semantic relevance. The example below demonstrates how to integrate GreenNodeRerank with a base retriever to improve the quality of retrieved documents.

In [11]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

from langchain_greennode import GreenNodeEmbeddings

# Initialize the embeddings model
embeddings = GreenNodeEmbeddings(
    # api_key="YOUR_API_KEY",  # You can pass the API key directly
    model="BAAI/bge-m3"  # The default embedding model
)

# Prepare documents (finance/economics domain)
docs = [
    Document(
        page_content="Inflation represents the rate at which the general level of prices for goods and services rises"
    ),
    Document(
        page_content="Central banks use interest rates to control inflation and stabilize the economy"
    ),
    Document(
        page_content="Cryptocurrencies like Bitcoin operate on decentralized blockchain networks"
    ),
    Document(
        page_content="Stock markets are influenced by corporate earnings, investor sentiment, and economic indicators"
    ),
]

# Create a vector store and a base retriever
vector_store = FAISS.from_documents(docs, embeddings)
base_retriever = vector_store.as_retriever(search_kwargs={"k": 4})


rerank_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=base_retriever
)

# Perform retrieval with reranking
query = "How do central banks fight rising prices?"
results = rerank_retriever.get_relevant_documents(query)

print("Reranked results for query:", query)
for i, doc in enumerate(results):
    print(f"Result {i+1} with score: {round(doc.metadata['relevance_score'], 5)}: {doc.page_content}")

Reranked results for query: How do central banks fight rising prices?
Result 1 with score: 0.125: Central banks use interest rates to control inflation and stabilize the economy
Result 2 with score: 0.00491: Inflation represents the rate at which the general level of prices for goods and services rises
Result 3 with score: 2e-05: Cryptocurrencies like Bitcoin operate on decentralized blockchain networks


## Direct Usage

The `GreenNodeRerank` class can be used independently to perform reranking of retrieved documents based on relevance scores. This functionality is particularly useful in scenarios where a primary retrieval step (e.g., keyword or vector search) returns a broad set of candidates, and a secondary model is needed to refine the results using more sophisticated semantic understanding. The class accepts a query and a list of candidate documents and returns a reordered list based on predicted relevance.

In [13]:
test_documents = [
    Document(page_content="Carson City is the capital city of the American state of Nevada."),
    Document(page_content="Washington, D.C. (also known as simply Washington or D.C.) is the capital of the United States.",),
    Document(page_content="Capital punishment has existed in the United States since beforethe United States was a country."),
    Document(page_content="The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan."),
]

test_query = "What is the capital of the United States?"
results = reranker.rerank(test_documents, test_query)
for i, res in enumerate(results):
    print(f"Result {i+1} with score: {round(res['relevance_score'], 5)}: {test_documents[res['index']].page_content}")

Result 1 with score: 1.0: Washington, D.C. (also known as simply Washington or D.C.) is the capital of the United States.
Result 2 with score: 0.01166: Carson City is the capital city of the American state of Nevada.
Result 3 with score: 0.00121: The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.


## API Reference

For more details about the GreenNode Serverless AI API, visit the [GreenNode Serverless AI Documentation](https://aiplatform.console.greennode.ai/api-docs/maas).